In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings(action='ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e9/sample_submission.csv
/kaggle/input/playground-series-s4e9/train.csv
/kaggle/input/playground-series-s4e9/test.csv


In [40]:
train = pd.read_csv("/kaggle/input/playground-series-s4e9/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e9/test.csv")

test_id = test['id']
test = test.drop('id', axis=1)

# ***PREPROCESSING***

In [41]:
X = train.drop(['price', 'id'], axis=1)
y = train['price']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline 

# subset the data 
numerical = X.select_dtypes(include='int64').columns
categorical = X.select_dtypes(include='object').columns

num_pipeline = Pipeline(steps=[
    ('mean_impute', SimpleImputer(strategy='mean'))
])

cat_pipeline = Pipeline(steps=[
    ('mode_impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore')),
])

transformer = ColumnTransformer(transformers=[
    ('num', num_pipeline, numerical),
    ('cat', cat_pipeline, categorical)])

X = transformer.fit_transform(X)
test = transformer.transform(test)

# ***SELECTION***

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

model = LinearRegression()
cv_score = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv=10)

rmse = -cv_score.mean()
rmse

75295.81708011522

In [51]:
model.fit(X, y) 
y_pred = model.predict(test)

submission = pd.DataFrame({'id': test_id, 'price': y_pred})

In [54]:
submission.to_csv('submission.csv', index=False)